In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from time import sleep
import argparse
parser = argparse.ArgumentParser(description="A simple command-line argument parser example.")
import os
import json

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='7'
llm = LLM(model='../model/Mistral-7B-Instruct-v0.2/',
        tensor_parallel_size=1,
        #   enable_chunked_prefill=False,
        enforce_eager=True,
        # gpu_memory_utilization=0.4,
        #   num_scheduler_steps=5,  
        #   enable_prefix_caching=False
        )  # Create an LLM.

WARNING 09-11 00:13:53 config.py:378] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 09-11 00:13:53 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='../model/Mistral-7B-Instruct-v0.2/', speculative_config=None, tokenizer='../model/Mistral-7B-Instruct-v0.2/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=

[W911 00:13:53.965577089 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [12-43]:52871 (errno: 97 - Address family not supported by protocol).


Loading pt checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/vllm/model_executor/model_loader/weight_utils.py:416: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state 

INFO 09-11 00:14:02 model_runner.py:926] Loading model weights took 13.4966 GB
INFO 09-11 00:14:04 gpu_executor.py:122] # GPU blocks: 27530, # CPU blocks: 2048


In [3]:
text_list = ['Who is George Bush?']
text_all = ["[INST] %s [/INST]" % str(a) for a in text_list]

In [4]:
sampling_params = SamplingParams(temperature=0, top_p=1,max_tokens=32,logprobs=1)
# print(format_dict)
outputs = llm.generate(text_all, 
                       sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s, est. speed input: 28.99 toks/s, output: 71.35 toks/s]


In [7]:
element_dict[key].decoded_token

'9'

In [5]:
generation_list = []
token_list_all = []
logprob_list_all = []
cumulative_logprob_all = []
# Print the outputs.
for output in outputs:
    logprob_lists = []
    prompt = output.prompt
    generated_text = output.outputs[0].text
    token_list = list(output.outputs[0].token_ids)
    logprob_list = []
    for element_dict in output.outputs[0].logprobs:
        
        key = list(element_dict.keys())[0]
        logprob_value = element_dict[key].logprob
        logprob_list.append(logprob_value)
    cumulative_logprob_all.append(output.outputs[0].cumulative_logprob)
    token_list_all.append(token_list)
    logprob_list_all.append(logprob_list)
    # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    generation_list.append(generated_text)
    

In [30]:
sum(logprob_list_all[0]),cumulative_logprob_all

(-1.371503817272469, [-1.371503817272469])

In [5]:
generation_list = []
logprob_list_all = []
# Print the outputs.
for output in outputs:
    logprob_lists = []
    prompt = output.prompt
    generated_text = output.outputs[0].text
    output
    # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    generation_list.append(generated_text)

In [12]:
token_list = list(output.outputs[0].token_ids)
for element_dict in output.outputs[0].logprobs:
    logprob_list = []
    key = list(element_dict.keys())[0]
    logprob_value = element_dict[key].logprob
    logprob_list.append(logprob_value)
    

5163 -9.786603914108127e-05
13668 -1.07287787614041e-05
349 -0.07919662445783615
396 -0.003101659705862403
2556 -0.000867467257194221
17587 -0.24034476280212402
693 -0.20168443024158478
6117 -9.119095193454996e-05
390 -2.7418097943154862e-06
272 -1.9073468138230965e-06
28705 -1.7881233361549675e-05
28781 -5.960462772236497e-07
28740 -0.11282405257225037
303 -3.576278118089249e-07
5120 -0.0013892533024773002
302 -3.576278118089249e-07
272 -9.536738616588991e-07
2969 -9.536697689327411e-06
3543 -6.651657167822123e-05
477 -6.437094270950183e-05
4624 -0.6935058236122131
28705 0.0
28740 -0.038041528314352036
28774 -1.0728830375228426e-06
28783 -2.9802276912960224e-06
28774 -2.3841855067985307e-07
298 -4.529942543740617e-06
4624 -0.0001731960946926847
28705 0.0
28740 0.0
28774 -9.536738616588991e-07
28774 -2.3841855067985307e-07


In [ ]:
def extract_logprobs(request_outputs):
    logprob_lists = []
    for request_output in request_outputs:
        for completion_output in request_output.outputs:
            token_logprobs = [logprob.logprob for logprob_dict in completion_output.logprobs for logprob in logprob_dict.values()]
            logprob_lists.append(token_logprobs)
    return logprob_lists
extract_logprobs(outputs)

In [ ]:
outputs